# AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması

## İş Problemi

Facebook kısa süre önce mevcut "maximum bidding" adı verilen 
teklif verme türüne alternatif olarak yeni bir teklif türü olan 
"average bidding"’i tanıttı. 

Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test 
etmeye karar verdi ve average bidding'in maximum bidding'den
daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B 
testi yapmak istiyor.

A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden 
bu A/B testinin sonuçlarını analiz etmenizi bekliyor.
Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu 
nedenle, istatistiksel testler için Purchase metriğine 
odaklanılmalıdır.


## Veri Seti Hikayesi

Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average Bidding uygulanmıştır.

    4 Değişken 40 Gözlem 26 KB

    Impression : Reklam görüntüleme sayısı
    Click      : Görüntülenen reklama tıklama sayısı
    Purchase   : Tıklanan reklamlar sonrası satın alınan ürün sayısı
    Earning    : Satın alınan ürünler sonrası elde edilen kazanç



## Proje Görevleri

### Görev 1: Veriyi Hazırlama ve Analiz Etme

Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı  değişkenlere atayınız.

Adım 2: Kontrol ve test grubu verilerini analiz ediniz.

Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz. 

    
### Görev 2: A/B Testinin Hipotezinin Tanımlanması
    
Adım 1: Hipotezi tanımlayınız.
    H0 : M1 = M2 
    H1 : M1!= M2 

Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.


### Görev 3: Hipotez Testinin Gerçekleştirilmesi    

Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.
Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni 
üzerinden ayrı ayrı test ediniz.
    Normallik Varsayımı :
    H0: Normal dağılım varsayımı sağlanmaktadır. 
    H1: Normal dağılım varsayımı sağlanmamaktadır.
    p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
    Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? Elde edilen p-value değerlerini yorumlayınız. 

Varyans Homojenliği :
    H0: Varyanslar homojendir.
    H1: Varyanslar homojen Değildir.
    p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
    Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediniz. 
    Test sonucuna göre normallik varsayımı sağlanıyor mu? Elde edilen p-value değerlerini yorumlayınız. 

Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki 
olarak anlamlı bir fark olup olmadığını yorumlayınız.


### Görev 4: Sonuçların Analizi

Adım 1: Hangi testi kullandınız, sebeplerini belirtiniz.

Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.




In [2]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [24]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# !pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

### Görev 1: Veriyi Hazırlama ve Analiz Etme

Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı  değişkenlere atayınız.

Adım 2: Kontrol ve test grubu verilerini analiz ediniz.

Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz. 


In [9]:
path="C:\\Users\\Msı\\PycharmProjects\\DSMLBC9\\W3_Measurement_Problems\\W3_projelerim\\W3_Proje_1_AB_Testi\\ab_testing.xlsx"
df_test=pd.read_excel(path,index_col=0, comment='#',sheet_name='Test Group') 
df_test

,Click,Purchase,Earning
Impression,,,
120103.50380,3216.54796,702.16035,1939.61124
134775.94336,3635.08242,834.05429,2929.40582
107806.62079,3057.14356,422.93426,2526.24488
116445.27553,4650.47391,429.03353,2281.42857
145082.51684,5201.38772,749.86044,2781.69752
115923.00695,4213.86862,778.37316,2157.40855
106116.43664,3279.47297,491.61453,2560.41120
125957.11610,4690.56991,855.71980,2563.57976
117442.86465,3907.93924,660.47791,2242.23259


In [10]:
df_control=pd.read_excel(path,index_col=0, comment='#',sheet_name='Control Group') 
df_control

,Click,Purchase,Earning
Impression,,,
82529.45927,6090.07732,665.21125,2311.27714
98050.45193,3382.86179,315.08489,1742.80686
82696.02355,4167.96575,458.08374,1797.82745
109914.40040,4910.88224,487.09077,1696.22918
108457.76263,5987.65581,441.03405,1543.72018
77773.63390,4462.20659,519.66966,2081.85185
95110.58627,3555.58067,512.92875,1815.00661
106649.18307,4358.02704,747.02012,1965.10040
122709.71659,5091.55896,745.98568,1651.66299


In [12]:
df_test["Purchase"].mean()/df_test["Click"].mean()

0.1467167727545302

In [11]:
df_control["Purchase"].mean()/df_control["Click"].mean()

0.10800452148227199

In [ ]:
# test grubunda satın alma otalaması daha yüksek gözüküyor
# test mi control grubu mu olduklarınıcyeni değişken atayarak belirtiyorum

In [18]:
df_test["test"]=1
df_test.head(2)

,Click,Purchase,Earning,test
Impression,,,,
120103.50380,3216.54796,702.16035,1939.61124,1
134775.94336,3635.08242,834.05429,2929.40582,1


In [19]:
df_control["test"]=0
df_control.head(2)

,Click,Purchase,Earning,test
Impression,,,,
82529.45927,6090.07732,665.21125,2311.27714,0
98050.45193,3382.86179,315.08489,1742.80686,0


In [20]:
df=pd.concat([df_test, df_control], ignore_index=False)
df

,Click,Purchase,Earning,test
Impression,,,,
120103.50380,3216.54796,702.16035,1939.61124,1
134775.94336,3635.08242,834.05429,2929.40582,1
107806.62079,3057.14356,422.93426,2526.24488,1
116445.27553,4650.47391,429.03353,2281.42857,1
145082.51684,5201.38772,749.86044,2781.69752,1
...,...,...,...,...
132064.21900,3747.15754,551.07241,2256.97559,0
86409.94180,4608.25621,345.04603,1781.35769,0
123678.93423,3649.07379,476.16813,2187.72122,0


### Görev 2: A/B Testinin Hipotezinin Tanımlanması
    
Adım 1: Hipotezi tanımlayınız.

    H0 : M1 = M2 
    H1 : M1!= M2 

Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.


In [ ]:
H0=Kontrol ve test grubu için satın alma değerleri arasında istatisliksel olarak anlamlı fark yoktur

In [21]:
df.groupby("test").agg({"Purchase": "mean"})

,Purchase
test,
0,550.89406
1,582.10610


### Görev 3: Hipotez Testinin Gerçekleştirilmesi    

Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.
Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni üzerinden ayrı ayrı test ediniz.

    Normallik Varsayımı :
    H0: Normal dağılım varsayımı sağlanmaktadır. 
    H1: Normal dağılım varsayımı sağlanmamaktadır.
    p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
    Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? Elde edilen p-value değerlerini yorumlayınız. 

Varyans Homojenliği :

    H0: Varyanslar homojendir.
    H1: Varyanslar homojen Değildir.
    p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
    Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediniz. 
    Test sonucuna göre normallik varsayımı sağlanıyor mu? Elde edilen p-value değerlerini yorumlayınız. 

Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki olarak anlamlı bir fark olup olmadığını yorumlayınız.


In [31]:
   
######################################################
# AB Testing (Bağımsız İki Örneklem T Testi)
######################################################
# iki grup ortalaması karşılaştırılmak istenildiğinde kullanılır.
### Grup karşılaştırmalarında temel amaç olası farklılıkların şans eseri ortaya çıkıp çıkmadığını göstermeye çalışmaktır. 
### iki grup birbirine eşit mi değil mi ona bakılır
### Örnek sayıları aynı,   varyanslar homejen ise,
### Örnek sayıları farklı, varyanslar homejen ise,
### Örnek sayıları farklı, varyanslar homejen değil ise, farklı kullanılacak formülsayon farklıdır.


# 1. Hipotezleri Kur (Ho=iki ortalama arasında istetisliki olarak anlamlı farklılık yoktur.)
# 2. Varsayım Kontrolü
#   - 1. Normallik Varsayımı
#   - 2. Varyans Homojenliği
# 3. Hipotezin Uygulanması
#   - 1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
#   - 2. Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)
# 4. p-value değerine göre sonuçları yorumla(p_valu<0.05 ise Ho hipotezi reddedilir)
# Not:
# - Normallik sağlanmıyorsa direk 2 numara. Varyans homojenliği sağlanmıyorsa 1 numaraya arguman girilir.
# - Normallik incelemesi öncesi aykırı değer incelemesi ve düzeltmesi yapmak faydalı olabilir.
    
    
    
# #### ÖNEMLİ NOT
# **ttest_ind** : Şu durumlarda kullanılır
# 1. Normallik varsayımı sağlanıyorsa
# 2. Normallik varsayımı sağlanıyor, Varsayımlar sağlanıyorsa (equal_var=True yapılır)
# 3. Normallik varsayımı sağlanıyor, Varsayımlar sağlanmıyorsa (equal_var=False yapılır)

# **mannwhitneyu testi**: non-parametrik ortalama/medyan kıyaslama testidir. ortalamalar arasında istatisliki olarak anlamlı bir farklılık var mı diye bakar.

# **NOT**: Hipotez sonuçlarına göre:
# 1. Ho reddedilir 
# 2. Ho reddedilemez
# **xxxxx "H1 kabul veya red edilemez" xxxxx buna aykırı bir sonuç çıkarılamaz. bu yanlıştır. H1 ile alakalı birşey diyemeyiz**    


#def bagimsiz_iki_orneklem_t_testi(df,col,M1,M2):
def bagimsiz_iki_orneklem_t_testi(df,col_x,col_y,M1,M2):    
    #     örneğin 
    #     col_x="smoker"
    #     col_y="total_bill"
    #     M1="Yes"
    #     M2="No"
    
    
    
    print(df.head())
    print("*"*99)
    print(df.groupby(col_x).agg({col_y: "mean"}))
    print("*"*99)
    
    df=df[[col_x,col_y]].dropna()
    
    par_test_pvalue,non_par_test_pvalue=999,999 #sadece hata vermesin diye değer atadık
    print("""
    ############################
    # 1. Hipotezi Kur
    ############################

    # H0: M1 = M2  (p-value !< 0.05, HO REDDEDİLEMEZ.)(Ortalamalar Arasında İstatistiksel Olarak Anlamlı Farklılık Yoktur) 
    # H1: M1! = M2 (p-value < 0.05,  HO RED.)         (........................................................... vardır) 
    
        # p-value < ise 0.05'ten HO RED.
        # p-value < değilse 0.05 H0 REDDEDILEMEZ.

    ############################
    # 2. Varsayım Kontrolü
    ############################

    # Normallik Varsayımı
        # H0: Normal dağılım varsayımı sağlanmaktadır.
    # Varyans Homojenliği
        # H0: Varyanslar homejendir.
        
        
    ############################
    # 3 ve 4. Hipotezin Uygulanması
    ############################

    # 1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
    # 2. Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)(medyan kıyası da yapılabilir)

    """)
    
    print("#"*30,"Normallik_Varsayimi","#"*30)
    test_stat, nv1_pvalue = shapiro(df.loc[df[col_x] == M1, col_y])
    print('Test Stat = %.4f, nv1_pvalue = %.4f' % (test_stat, nv1_pvalue))
    
    test_stat, nv2_pvalue = shapiro(df.loc[df[col_x] == M2, col_y])
    print('Test Stat = %.4f, nv2_pvalue = %.4f' % (test_stat, nv2_pvalue))
    print("*"*99)
    
    if nv1_pvalue<0.05 or nv2_pvalue<0.05 :
        
        print("HO RED. Normallik_Varsayimi sağlanmadı. Non parametrik test uygulanmalı")
        print("       2. Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)")
        print("*"*99)


        ############################
        # 1.2 Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)
        ############################
        # p-value < ise 0.05 'ten HO RED.
        # p-value < değilse 0.05 H0 REDDEDILEMEZ.
        
        test_stat, non_par_test_pvalue = mannwhitneyu(df.loc[df[col_x] == M1, col_y],
                                                      df.loc[df[col_x] == M2, col_y])

        print('Test Stat = %.4f, non_par_test_pvalue = %.4f' % (test_stat, non_par_test_pvalue))
        print("*"*99)
        
    else:
        print("HO REDDEDILEMEZ.Normallik_Varsayimi sağlandı. parametrik test uygulanmalı.")
        print("    1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)")
        print("*"*99)
        ############################
        # Varyans Homojenligi Varsayımı
        ############################

        # H0: Varyanslar Homojendir
        # H1: Varyanslar Homojen Değildir
        
        # p-value < ise 0.05 'ten HO RED.
        # p-value < değilse 0.05 H0 REDDEDILEMEZ.
        
        
        print("#"*30,"Varyans Homojenligi","#"*30)
        test_stat, vh_pvalue = levene(df.loc[df[col_x] == M1, col_y],
                                      df.loc[df[col_x] == M2, col_y])
        print('    Test Stat = %.4f, vh_pvalue = %.4f' % (test_stat, vh_pvalue))

        if vh_pvalue<0.05 :
            print("HO RED.  Varyans Homojenligi Varsayımı sağlanmadı.")
            equal_var_degeri=False
        else:
            print("HO REDDEDILEMEZ.Varyans Homojenligi Varsayımı sağlandı.")
            equal_var_degeri=True
    
        print("*"*99)
        ############################
        # 1.1 Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
        ############################
        # p-value < ise 0.05 'ten HO RED.
        # p-value < değilse 0.05 H0 REDDEDILEMEZ.
        
        
        print("#"*30,"H0 Hipotezi","#"*30)
        test_stat, par_test_pvalue = ttest_ind(df.loc[df[col_x] == M1, col_y],
                                      df.loc[df[col_x] == M2, col_y],
                                      equal_var=equal_var_degeri)

        print('    Test Stat = %.4f, par_test_pvalue = %.4f' % (test_stat, par_test_pvalue))
        print("*"*99)
    if  par_test_pvalue < 0.05 or non_par_test_pvalue < 0.05:
        print("""
            pvalue < 0.05
            H0 RED.//// #hipotezimiz>>> H0: M1 = M2
            # sonuç : iki ortalama arasında istatisliki olarak anlamlı bir farklılık vardır.
            """)
    else:
        print("""
            pvalue !< 0.05
            H0 REDDEDILEMEZ.//// #hipotezimiz>>> H0: M1 = M2
            # sonuç : iki ortalama arasında istatisliki olarak anlamlı bir farklılık yoktur.
            """)
        

In [32]:
bagimsiz_iki_orneklem_t_testi(df,col_x="test",col_y="Purchase",M1=1,M2=0)

                  Click  Purchase    Earning  test
Impression                                        
120103.50380 3216.54796 702.16035 1939.61124     1
134775.94336 3635.08242 834.05429 2929.40582     1
107806.62079 3057.14356 422.93426 2526.24488     1
116445.27553 4650.47391 429.03353 2281.42857     1
145082.51684 5201.38772 749.86044 2781.69752     1
***************************************************************************************************
      Purchase
test          
0    550.89406
1    582.10610
***************************************************************************************************

    ############################
    # 1. Hipotezi Kur
    ############################

    # H0: M1 = M2  (p-value !< 0.05, HO REDDEDİLEMEZ.)(Ortalamalar Arasında İstatistiksel Olarak Anlamlı Farklılık Yoktur) 
    # H1: M1! = M2 (p-value < 0.05,  HO RED.)         (........................................................... vardır) 
    
        # p-value < ise 0.05'ten HO RED.

### Görev 4: Sonuçların Analizi

Adım 1: Hangi testi kullandınız, sebeplerini belirtiniz.

Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.


In [ ]:
# Normallik_Varsayimi için shapiro testini(Bağımsız İki Örneklem T Testi'nde Normallik_Varsayimi için shapiro kullanılır),
# Varyans Homojenligi levene testini(Bağımsız İki Örneklem T Testi'nde Varyans Homojenligi için levene kullanılır),
# Normallik_Varsayimi ve Varyans Homojenligi sağlandığı için parametrik test olan ttest_ind testini kullandım.

In [33]:
df.groupby("test").agg({"Purchase": "mean"})

,Purchase
test,
0,550.89406
1,582.10610


In [ ]:
# Yapılan çalışma sonucunda iki ortalama arasında istatisliki olarak anlamlı bir farklılık yoktur.
# Ayrıca görüldüğü üzere kontrol grubu ortalamaları normal ortalamaların altında kalmıştır. 
# bu durumda ortalamalar arasında fark olsa bile bu durum yapılan çalışmanın aleyhine olacaktı. 
# Dolayısı ile yapılan çalışma kazanç sağlamadığı açıktır. hatta istatisliki olarak kesin olmamakla birlikte zarara sebep olma ihtimali bile vardır.